# unittest.mock
## Basic methods

### Create a Mock object

In [2]:
from unittest import mock
m = mock.Mock()

### Set a default return value when calling mock object

In [3]:
m.return_value = 42
m()

42

### Assign different return values for successive calls

``side_effect`` must be of type list. Each time the mocked method is then called,
the next value in the list is returned. If the ``side_effect`` list has no more
values to return, the next call will fail with a ``StopIteration`` error.

The ``return_value`` is like a constant side_effect, which always returns the same value.

In [4]:
m.side_effect = ['foo', 'bar', 'baz']
print(m())
print(m())
print(m())

foo
bar
baz


### Check whether mock object has been called at least once

In [5]:
m = mock.Mock()
m()
m.assert_called()

Returns *None*, if called, otherwise raises *AssertionError*

### Check whether mock object has been called exactly once

In [6]:
m = mock.Mock()
try:
    m.assert_called_once()
except AssertionError:
    print("No, I wasn't called (yet)")
m()
if not m.assert_called_once():
    print("Yeah, now I was called")

No, I wasn't called (yet)
Yeah, now I was called


### Get number of calls to mock object

In [7]:
m = mock.Mock()
m()
m()
m.call_count

2

### Get call arguments of last mock call

In [8]:
m = mock.Mock()
m(1, foo='bar')
m.call_args

call(1, foo='bar')

If mock object hasn't been called yet, *None* is returned.

### Get call arguments of all mock calls

In [9]:
m = mock.Mock()
m()
m(1, foo='bar')
m(4, baz='bar2')
m.call_args_list

[call(), call(1, foo='bar'), call(4, baz='bar2')]

### Reset call previous call (won't change mock configuration)

In [10]:
m = mock.Mock()
m()
m(1, foo='bar')
m(4, baz='bar2')
m.reset_mock()
m.call_args_list

[]

## Patching an import module
``unittest.mock`` is able to patch imports in the module under test using the
[patch](https://docs.python.org/3/library/unittest.mock.html#patch)` function.

``patch`` will intercept import statements identified by a string and return
a Mock instance you can preconfigure using the techniques from above.

Module under test (``work.py``):

```python
import os

def work_on():
    path = os.getcwd()
    print(f'Working on {path}')
    return path
```

The project module imports ``os`` and uses its ``getcwd()`` method , which
we want to mock in our test.

### Patch using a context manager (``with`` statement)

When using the context manager ``patch``, the patch ends when the ``with`` statement ends.

```python
from unittest import TestCase, mock
from work import work_on

class TestWorkMockingModule(TestCase):

    def test_using_context_manager(self):
        with mock.patch('work.os') as mocked_os:
            work_on()
            mocked_os.getcwd.assert_called_once()
```

We specifically patch the ``work.os`` module, not ``os``, as this
would patch ``os`` for all modules, not just for ``work.py``.

* The test module imports the project module's method ``work_on()``
* At the beginning of the test, the ``work.os`` module (which is the ``os``
  module in our ``work.py`` module) is patched with a [MagickMock][1] object
  (here called ``mocked_os``)
* When the ``work_on()`` method is called afterward, that ``MagicMock`` object is
  called instead of the original module (``os``) - as no return value is defined
  ``work_on()`` returns another MagicMock object - as we are not testing the
  correctness of the return value here, only checking if the ``mocked_os`` executed
  the ``getcwd()``method, we do not care about the return value

The ``as`` statement in the decorator is optional. You may as well not mock the
entire ``work.os`` module, but rather define a return value for the single
function, that is called (``getcwd()``):

```python
from unittest import TestCase, mock
from work import work_on

class TestWorkMockingModule(TestCase):
    def test_using_return_value(self):
        with mock.patch('work.os.getcwd', return_value='testing'):
            assert work_on() == 'testing'
```

### Patch using the decorator (@patch)
Using the ``@patch`` decorator allows to inject a mock into the test function.
The patch is available for the entire function.

```python
from unittest import TestCase, mock

from work import work_on

class TestWorkMockingModule(TestCase):

    @mock.patch('work.os')
    def test_using_decorator(self, mocked_os):
        work_on()
        mocked_os.getcwd.assert_called_once()
```

Here the ``work.os`` module is replaced with a MagicMock object.

[1]: https://docs.python.org/3/library/unittest.mock.html?highlight=magicmock#unittest.mock.MagicMock

## Patching classes
To test project classes that interact with other project classes in isolation,
the other project classes must be mocked (in order to determine test failure
to a specific class and method, not by any of its dependency classes).

Example:

### Project module (worker.py)

In this module, the `Worker` class needs to be tested for these two things:

- the `Worker` calls `Helper` with `db`
- the `Worker` returns the expected path supplied by `Helper `

```python
import os


class Helper:

    def __init__(self, path):
        self.path = path

    def get_path(self):
        base_path = os.getcwd()
        return os.path.join(base_path, self.path)


class Worker:

    def __init__(self):
        self.helper = Helper('db')

    def work(self):
        path = self.helper.get_path()
        print(f'Working on {path}')
        return path
```

To test this, the `Worker` needs to be isolated from `Helper`,
so that failures can be associated with the `Worker` class only.

Consequently, the entire `Helper` class must be patched:

```python
from unittest import TestCase, mock
from worker import Worker


class TestWorkerModule(TestCase):

    def test_patching_class(self):
        with mock.patch('worker.Helper') as MockHelper:
            MockHelper.return_value.get_path.return_value = 'testing'
            worker = Worker()
            MockHelper.assert_called_once_with('db')
            self.assertEqual(worker.work(), 'testing')
```

Note the double ``return_value`` in the example, simply using 
``MockHelper.get_path.return_value`` would not work since in the code
we call get_path on an instance, not the class itself.

Alternatively, we can preconfigure the ``MockHelper`` instance and
assign it as the return value of the ``worker.Helper`` call. Below,
shown for both the context manager and the decorator way:

```python
from unittest import TestCase, mock
from worker import Worker


class TestWorkerModule(TestCase):

    mockHelperInstance = mock.MagicMock()
    mockHelperInstance.get_path.return_value = 'testing'

    def test_patch_class_context_manager(self):
        with mock.patch('worker.Helper',
                        return_value=self.mockHelperInstance) as MockHelper:
            worker = Worker()
            MockHelper.assert_called_once_with('db')
            self.assertEqual(worker.work(), 'testing')

    @mock.patch('worker.Helper', return_value=mockHelperInstance)
    def test_patch_class_decorator(self, mock_helper):
        worker = Worker()
        mock_helper.assert_called_once_with('db')
        self.assertEqual(worker.work(), 'testing')
```


*NEEDS TO BE CONTINUED...*